# 02 - Análisis Exploratorio de Datos (EDA)

**Objetivo:** Explorar y visualizar los datos para entender patrones, correlaciones y problemas.

**Input:** `data/processed/dataset_modelo.csv`

**Output:** Gráficos en `data/processed/`

## 1. Setup

In [ ]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Directorio de trabajo
os.chdir("/Users/javiermondragon/Documents/data_projects/inflation-predictor")

print("Setup completo")

In [ ]:
# Cargar datos
df = pd.read_csv('data/processed/dataset_modelo.csv', parse_dates=['date'])

print(f"Dataset cargado: {len(df)} filas")
print(f"Período: {df['date'].min().strftime('%Y-%m')} a {df['date'].max().strftime('%Y-%m')}")
print(f"Columnas: {list(df.columns)}")

## 2. Estadísticas Descriptivas

In [ ]:
# Variables numéricas
variables = ['cpi', 'fed_rate', 'oil_price', 'gold_price']

print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)
df[variables].describe().round(2)

In [ ]:
# Estadísticas adicionales
print("\nESTADÍSTICAS ADICIONALES")
print("=" * 60)

for var in variables:
    print(f"\n{var.upper()}")
    print(f"  Mediana: {df[var].median():.2f}")
    print(f"  Asimetría (skew): {df[var].skew():.2f}")
    print(f"  Curtosis: {df[var].kurtosis():.2f}")
    print(f"  Coef. variación: {(df[var].std() / df[var].mean() * 100):.1f}%")

## 3. Distribuciones

In [ ]:
# Histogramas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colores = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']

for ax, var, color in zip(axes.flatten(), variables, colores):
    ax.hist(df[var], bins=30, color=color, edgecolor='white', alpha=0.8)
    ax.set_title(f'Distribución de {var.upper()}', fontsize=12, fontweight='bold')
    ax.set_xlabel(var)
    ax.set_ylabel('Frecuencia')
    
    # Líneas de media y mediana
    media = df[var].mean()
    mediana = df[var].median()
    ax.axvline(media, color='black', linestyle='-', linewidth=2, label=f'Media: {media:.1f}')
    ax.axvline(mediana, color='gray', linestyle='--', linewidth=2, label=f'Mediana: {mediana:.1f}')
    ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('data/processed/eda_01_distribuciones.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_01_distribuciones.png")

## 4. Series de Tiempo

In [ ]:
# Series de tiempo
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

for ax, var, color in zip(axes, variables, colores):
    ax.plot(df['date'], df[var], color=color, linewidth=1.5)
    ax.set_ylabel(var.upper(), fontsize=10, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.fill_between(df['date'], df[var], alpha=0.2, color=color)

axes[0].set_title('Series de Tiempo - Variables del Modelo', fontsize=14, fontweight='bold')
axes[-1].set_xlabel('Fecha', fontsize=11)

plt.tight_layout()
plt.savefig('data/processed/eda_02_series_tiempo.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_02_series_tiempo.png")

## 5. Correlaciones

In [ ]:
# Matriz de correlación
corr_matrix = df[variables].corr()

print("MATRIZ DE CORRELACIÓN")
print("=" * 60)
print(corr_matrix.round(3))

In [ ]:
# Heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(corr_matrix, 
            annot=True, 
            fmt='.3f', 
            cmap='RdYlBu_r',
            center=0,
            square=True,
            linewidths=1,
            ax=ax,
            annot_kws={'size': 14},
            vmin=-1, vmax=1)

ax.set_title('Matriz de Correlación', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('data/processed/eda_03_correlacion.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_03_correlacion.png")

## 6. Relación con CPI (Target)

In [ ]:
# Scatter plots vs CPI
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

predictores = ['fed_rate', 'oil_price', 'gold_price']
colores_pred = ['#A23B72', '#F18F01', '#C73E1D']

for ax, var, color in zip(axes, predictores, colores_pred):
    ax.scatter(df[var], df['cpi'], alpha=0.6, c=color, s=50)
    ax.set_xlabel(var.upper(), fontsize=11)
    ax.set_ylabel('CPI', fontsize=11)
    
    # Línea de tendencia
    z = np.polyfit(df[var], df['cpi'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df[var].min(), df[var].max(), 100)
    ax.plot(x_line, p(x_line), 'k--', linewidth=2, alpha=0.7)
    
    # Correlación
    corr = df['cpi'].corr(df[var])
    ax.set_title(f'{var.upper()} vs CPI\nr = {corr:.3f}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/processed/eda_04_scatter_cpi.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_04_scatter_cpi.png")

## 7. Detección de Outliers

In [ ]:
# Boxplots
fig, axes = plt.subplots(1, 4, figsize=(14, 5))

for ax, var, color in zip(axes, variables, colores):
    bp = ax.boxplot(df[var], patch_artist=True)
    bp['boxes'][0].set_facecolor(color)
    bp['boxes'][0].set_alpha(0.7)
    ax.set_title(var.upper(), fontsize=12, fontweight='bold')
    ax.set_ylabel('Valor')

plt.suptitle('Detección de Outliers (Boxplots)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('data/processed/eda_05_outliers.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_05_outliers.png")

In [ ]:
# Cuantificar outliers con IQR
print("DETECCIÓN DE OUTLIERS (método IQR)")
print("=" * 60)

for var in variables:
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df[(df[var] < lower) | (df[var] > upper)]
    
    print(f"\n{var.upper()}")
    print(f"  Rango normal: [{lower:.2f}, {upper:.2f}]")
    print(f"  Outliers: {len(outliers)} ({100*len(outliers)/len(df):.1f}%)")

## 8. Análisis de Inflación

In [ ]:
# Calcular tasas de inflación
df['cpi_pct_mom'] = df['cpi'].pct_change() * 100  # Month over month
df['cpi_pct_yoy'] = df['cpi'].pct_change(12) * 100  # Year over year

print("ESTADÍSTICAS DE INFLACIÓN")
print("=" * 60)
print(f"\nInflación MoM:")
print(f"  Media: {df['cpi_pct_mom'].mean():.2f}%")
print(f"  Máxima: {df['cpi_pct_mom'].max():.2f}%")
print(f"  Mínima: {df['cpi_pct_mom'].min():.2f}%")

print(f"\nInflación YoY:")
print(f"  Media: {df['cpi_pct_yoy'].mean():.2f}%")
print(f"  Máxima: {df['cpi_pct_yoy'].max():.2f}%")
print(f"  Mínima: {df['cpi_pct_yoy'].min():.2f}%")

In [ ]:
# Gráfico de inflación
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Inflación mes a mes
axes[0].bar(df['date'], df['cpi_pct_mom'], color='#2E86AB', alpha=0.7, width=20)
axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_ylabel('Inflación MoM (%)')
axes[0].set_title('Inflación Mes a Mes', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Inflación año a año
axes[1].plot(df['date'], df['cpi_pct_yoy'], color='#C73E1D', linewidth=2)
axes[1].fill_between(df['date'], df['cpi_pct_yoy'], alpha=0.3, color='#C73E1D')
axes[1].axhline(2, color='green', linewidth=2, linestyle='--', label='Meta Fed (2%)')
axes[1].set_ylabel('Inflación YoY (%)')
axes[1].set_xlabel('Fecha')
axes[1].set_title('Inflación Año a Año', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/processed/eda_06_inflacion.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_06_inflacion.png")

## 9. Pairplot Completo

In [ ]:
# Pairplot
g = sns.pairplot(df[variables], diag_kind='kde', plot_kws={'alpha': 0.6})
g.fig.suptitle('Relaciones entre Variables', y=1.02, fontsize=14, fontweight='bold')

plt.savefig('data/processed/eda_07_pairplot.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: eda_07_pairplot.png")

## 10. Resumen de Hallazgos

In [ ]:
print("="*60)
print("RESUMEN DE HALLAZGOS DEL EDA")
print("="*60)

print("\n1. CORRELACIONES CON CPI:")
for var in predictores:
    corr = df['cpi'].corr(df[var])
    fuerza = "fuerte" if abs(corr) > 0.7 else "moderada" if abs(corr) > 0.4 else "débil"
    direccion = "positiva" if corr > 0 else "negativa"
    print(f"   {var}: {corr:.3f} ({fuerza}, {direccion})")

print("\n2. DISTRIBUCIONES:")
for var in variables:
    skew = df[var].skew()
    tipo = "simétrica" if abs(skew) < 0.5 else "sesgada a la derecha" if skew > 0 else "sesgada a la izquierda"
    print(f"   {var}: {tipo} (skew={skew:.2f})")

print("\n3. OUTLIERS:")
for var in variables:
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[var] < Q1 - 1.5*IQR) | (df[var] > Q3 + 1.5*IQR)]
    print(f"   {var}: {len(outliers)} outliers ({100*len(outliers)/len(df):.1f}%)")

print("\n4. PERÍODO DE ANÁLISIS:")
print(f"   {df['date'].min().strftime('%Y-%m')} a {df['date'].max().strftime('%Y-%m')}")
print(f"   {len(df)} observaciones mensuales")

## Próximos Pasos

1. **Feature Engineering:** Crear lags, promedios móviles, variaciones
2. **Modelado:** Entrenar modelos de predicción
3. **Evaluación:** Walk-forward validation